# Crude Onshore Inventories Notebook

This notebook will give a tutorial on how to query the onshore crude inventories data through pythonSDK.


Content:
1. [Import Libraries](#Import-Libraries)
2. [Extract storage location ids](#Extract-storage-location-ids)
3. [Crude Inventories Search endpoint](#1.-Crude-Inventories-Search-endpoint)
4. [Crude Onshore Inventories Time Series](#2.-Crude-Onshore-Inventories-Time-Series-endpoint)
5. [Case study](#Case-study:-US-Cushing-Stockpile-vs-Brent-prices)
6. [Benchmarking result](#Benchmarking-result)

## Import Libraries

In [1]:
import vortexasdk as v
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import json
from pprint import pprint
import plotly.graph_objects as go

c:\Users\OuiWeinJien\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## 1. Crude Inventories Search endpoint

In [2]:
cushing = 'cde783a902c7837b814dfa5988ed74fb14841c2999d6984952c7cbfc543d5073'
china = [p.id for p in v.Geographies().search('China').to_list() if p.name=='China']

2024-07-30 10:32:15,377 vortexasdk.client — WARNING — You are using vortexasdk version 0.72.5, however version 0.72.6 is available.
You should consider upgrading via the 'pip install vortexasdk --upgrade' command.


In [7]:
data_df = v.OnshoreInventoriesSearch().search(
        crude_confidence=['confirmed', 'probable'],
        location_ids = china,
        time_min=datetime(2023, 3, 1),
        time_max=datetime(2023, 3, 14)
        ).to_df(columns = 'all')

In [11]:
data_df.columns

Index(['measurement_id', 'tank_id', 'tank_details.id', 'tank_details.lat',
       'tank_details.lon', 'tank_details.name', 'tank_details.leaf',
       'tank_details.ref_type', 'tank_details.storage_terminal_name',
       'tank_details.storage_terminal_id', 'tank_details.storage_type',
       'tank_details.radius', 'tank_details.capacity_bbl',
       'tank_details.capacity_cbm', 'tank_details.capacity_ton',
       'tank_details.crude_confidence', 'tank_details.last_updated',
       'tank_details.location_id', 'tank_details.location_details.0.id',
       'tank_details.location_details.0.label',
       'tank_details.location_details.0.layer',
       'tank_details.location_details.1.id',
       'tank_details.location_details.1.label',
       'tank_details.location_details.1.layer',
       'tank_details.location_details.2.id',
       'tank_details.location_details.2.label',
       'tank_details.location_details.2.layer',
       'tank_details.location_details.3.id',
       'tank_details.loca

In [5]:
data_list = v.OnshoreInventoriesSearch().search(
        crude_confidence=['confirmed', 'probable'],
        location_ids = china,
        time_min=datetime(2023, 3, 1),
        time_max=datetime(2023, 3, 14)).to_list()

In [6]:
pprint(data_list[0].dict())

{'fill_bbl': 53429,
 'fill_cbm': 8494.569851999999,
 'fill_tons': 7305.2937409999995,
 'measurement_id': '000e025d8be0f76d3fe6a23bbd6867436ff751ad8fe9d85f30846afcfc077d19',
 'measurement_timestamp': '2023-03-02T10:13:39',
 'publish_timestamp': '2023-03-03T18:00:01',
 'reference_data_version': 'current-1718644569619',
 'tank_details': {'capacity_bbl': 318884,
                  'capacity_cbm': 50698,
                  'capacity_ton': 43600,
                  'corporate_entity_details': {'id': 'f78558ef7c9b9f162545a56b330111c274d578a2933f46ec82ee004ca4f449bd',
                                               'label': 'PETROCHINA',
                                               'layer': 'commercial_owner'},
                  'corporate_entity_id': 'f78558ef7c9b9f162545a56b330111c274d578a2933f46ec82ee004ca4f449bd',
                  'crude_confidence': 'confirmed',
                  'id': '1432c91cc383424b9fa048a7cc3fbe87747aa5e575a3e2c65479da8a7fd630d0',
                  'last_updated': '20

## 2. Crude Onshore Inventories Time Series endpoint

In [7]:
shandong = [p.id for p in v.StorageTerminals().search(term ='Shandong').to_list() if p.name=='Shandong Province']

In [8]:
df = v.OnshoreInventoriesTimeseries().search(
        crude_confidence=['confirmed', 'probable'],
        location_ids = shandong,
        time_min=datetime(2022, 1, 1),
        time_max=datetime.today(),
        timeseries_frequency = 'day',
        timeseries_unit = 'b'
).to_df(columns = 'all').set_index('key')

df2 = v.OnshoreInventoriesTimeseries().search(
        crude_confidence=['confirmed', 'probable'],
        location_ids = china,
        time_min=datetime(2022, 1, 1),
        time_max=datetime.today(),
        timeseries_frequency = 'day',
        timeseries_unit = 'b'
).to_df(columns = 'all').set_index('key')

In [9]:
# Create traces
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=df.index, y=df['value'],
    line_color='rgb(0,100,80)',
    name='Shandong'),
    secondary_y=False
)

fig.add_trace(go.Scatter(
    x=df2.index, y=df2['value'],
    line_color='rgb(255,100,80)',
    name='china'),
    secondary_y=True
)

fig.update_layout(
    title_text="Shandong vs China Crude Inventories (bbls)"
)

# Set y-axes titles
fig.update_yaxes(title_text="Shandong volume", secondary_y=False)
fig.update_yaxes(title_text="China volume", secondary_y=True)

fig.show()

***Shandong Inventories decline but remain a crucial outlier***

Shandong crude stocks have been falling consistently sisnce mid last year, but are still well above year-earlier levels.

This contrasts with total China inventories now falling close to historical norms and global inventories currently at lows.

Shandong's independent refiners can continue to draw at current rates for the next 6 months and still remain above historical avearage.

## Case study: US Cushing Stockpile vs Brent prices

In [6]:
import yfinance as yf
price_history = yf.Ticker('BZ=Fdsdsds').history(period='2y', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                   interval='2d', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                   actions=False)

BZ=FDSDSDS: Invalid input - interval=2d is not supported. Valid intervals: , 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]


In [7]:
len(price_history)

0

In [12]:
df3 = v.OnshoreInventoriesTimeseries().search(
        crude_confidence=['confirmed', 'probable'],
        location_ids = cushing,
        time_min=datetime(2022, 1, 1),
        time_max=datetime.today(),
        timeseries_frequency = 'day',
        timeseries_unit = 'b'
).to_df(columns = 'all').set_index('key')

In [13]:
# Create traces
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=df3.index, y=df3['value'],
    line_color='rgb(0,100,80)',
    name='US Crude stockpiles'),
    secondary_y=False
)

fig.add_trace(go.Scatter(
    x=price_history.index, y=price_history['Open'],
    line_color='rgb(255,100,80)',
    name='Brent Price'),
    secondary_y=True
)

fig.update_layout(
    title_text="US Cushing stockpiles vs Brent Price"
)

# Set y-axes titles
fig.update_yaxes(title_text="Cushing volume (b)", secondary_y=False)
fig.update_yaxes(title_text="Brent Price ($)", secondary_y=True)

fig.show()

***Vortexa data as a leading indicator of inventories data as it publishes more than a weekly basis, which can be useful for price analysis.***

Inverse correlation is found for cushing volume vs Brent prices. As Vortexa publishes measurement readings for each tank whenever we receive it, the frequency in data is more competitive than publicly available data such as Energy Information Administration (EIA).